### Dataset Relation

`ais_tracks` & `radar_tracks`: both contain 27416 corresponding tracks identified by both radar & ais. Can consider them as training

`tracks_tagged`: 9013 tagged radar tracks by users in M2. 

`tagged_detections`: 6756272 timestamps and 9020 individual tracks in total, contains ALL tracks in `tracks_tagged`. The remaining 7 tracks are from AIS, 5 of which overlap with `ais_tracks`

`radar_detections`: 7387790 timestamps and 19947 radar tracks; 

### Initial Ideas:

`tracks_tagged`:
* notes have useful information that can be extracted



In [20]:
len(set(radar_detections['assoc_id']))

16591

In [14]:
len(set(radar_detections['assoc_id']) & set(ais_tracks['id_track']))

15345

In [15]:
import pandas as pd
import numpy as np

# Define file paths
ais_tracks_path = '../data/tracks_ais.csv'
radar_tracks_path = '../data/tracks_radar.csv'
radar_detections_path = '../data/detections_radar.csv'
tagged_detections_path = '../data/detections_tagged.csv'
tracks_tagged_path = '../data/tracks_tagged.csv'

ais_tracks = pd.read_csv(ais_tracks_path)
radar_tracks = pd.read_csv(radar_tracks_path)
radar_detections = pd.read_csv(radar_detections_path)
tagged_detections = pd.read_csv(tagged_detections_path)
tracks_tagged = pd.read_csv(tracks_tagged_path)

print("Length of ais_tracks:", len(ais_tracks))
print("Length of radar_tracks:", len(radar_tracks))
print("Length of radar_detections:", len(radar_detections))
print("Length of tagged_detections:", len(tagged_detections))
print("Length of tracks_tagged:", len(tracks_tagged))

Length of ais_tracks: 27416
Length of radar_tracks: 27416
Length of radar_detections: 7387790
Length of tagged_detections: 6756272
Length of tracks_tagged: 9013


In [16]:
ais_tracks_id = set(ais_tracks['id_track'])
radar_tracks_id = set(radar_tracks['id_track'])
radar_detections_id = set(radar_detections['id_track'])
tagged_detections_id = set(tagged_detections['id_track'])
tracks_tagged_id = set(tracks_tagged['id_track'])

print("Unique ais_tracks_id count:", len(ais_tracks_id))
print("Unique radar_tracks_id count:", len(radar_tracks_id))
print("Unique radar_detections_id count:", len(radar_detections_id))
print("Unique tagged_detections_id count:", len(tagged_detections_id))
print("Unique tracks_tagged_id count:", len(tracks_tagged_id))

Unique ais_tracks_id count: 27416
Unique radar_tracks_id count: 27416
Unique radar_detections_id count: 19947
Unique tagged_detections_id count: 9020
Unique tracks_tagged_id count: 9013


In [17]:
tracks_tagged.iloc[1]

id_track             30278317
id_site                    10
id_m2           158-0714-0950
source                  radar
duration                 5463
alarm                       1
min_speed                 0.1
max_speed                 6.7
avg_speed               3.016
curviness            2.591425
heading_mean      -124.436676
heading_std          1.784569
turning_mean         6.125324
turning_std         14.824189
duration_z               4399
distance             9.013697
distance_o           3.478278
assoc_str                 NaN
assoc_id                  NaN
tagged                      1
has_photos                  1
confidence           0.989689
detections              850.0
sdate              2023-07-14
stime                09:50:48
ldate              2023-07-14
ltime                11:21:51
user_id                   331
valid                       1
type                  unknown
notes                   Sail 
transit                     0
overnight                   0
loiter    

In [18]:
tagged_detections.head(4)

,id_detect,id_track,id_site,id_m2,source,speed,course,assoc_str,assoc_id,confidence,cdate,ctime,longitude,latitude
0,1315108673,29980760,10,18-0704-1000,radar,7.5,3.2,NaN,NaN,0.872091,2023-07-04,10:00:12,-117.279102,32.798307
1,1315108674,29980760,10,18-0704-1000,radar,11.3,349.5,NaN,NaN,0.872091,2023-07-04,10:00:18,-117.279330,32.798868
2,1315108675,29980760,10,18-0704-1000,radar,13.3,347.6,NaN,NaN,0.872091,2023-07-04,10:00:24,-117.279482,32.799303
3,1315108676,29980760,10,18-0704-1000,radar,13.6,351.3,NaN,NaN,0.872091,2023-07-04,10:00:30,-117.279515,32.799792
